In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker-nightly

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 0s (450 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype


In [ ]:
import numpy as np
import os

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [ ]:
!pip install gdown
!gdown --id 1fngJLkq2c9OJaryuWXtCBvBw0F9rBgV9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1fngJLkq2c9OJaryuWXtCBvBw0F9rBgV9
To: /content/Suicide_Detection.csv
100% 167M/167M [00:01<00:00, 90.8MB/s]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
csv = pd.read_csv('Suicide_Detection.csv').iloc[:,1:]

In [ ]:
train, test = train_test_split(csv, test_size=0.2)


In [ ]:
train.to_csv("train.csv")

In [ ]:
test.to_csv("test.csv")

In [ ]:
spec = model_spec.get('average_word_vec')

In [ ]:
mb_spec = model_spec.get('mobilebert_classifier')

In [ ]:
train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='text',
      label_column='class',
      model_spec=mb_spec,
      is_training=True)
test_data = DataLoader.from_csv(
      filename='test.csv',
      text_column='text',
      label_column='class',
      model_spec=mb_spec,
      is_training=False)

In [ ]:
model = text_classifier.create(train_data, model_spec=mb_spec, epochs=2)

3867/3867 [==============================] - 1735s 436ms/step - loss: 0.1956 - test_accuracy: 0.9375
Epoch 2/2
3867/3867 [==============================] - 1689s 436ms/step - loss: 0.0531 - test_accuracy: 0.9824


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 hub_keras_layer_v1v2_1 (HubKer  (None, 512)         24581888    ['input_word_ids[0][0]',         
 asLayerV1V2)                                                     'input_mask[0][0]',       

In [ ]:
loss, acc = model.evaluate(test_data)

1451/1451 [==============================] - 173s 117ms/step - loss: 0.0765 - test_accuracy: 0.9756


In [ ]:
model.export(export_dir='mobilebert/')